# Задание 1

In [1]:
import pandas as pd

In [3]:
log = pd.read_csv('visit_log.csv', sep = ';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [6]:
# если источник traffic_source равен yandex или google, то в source_type ставится organic
# для источников paid и email из России - ставим ad
# для источников paid и email не из России - ставим other
# все остальные варианты берем из traffic_source без изменений

log.loc[(log.traffic_source == 'yandex') | (log.traffic_source == 'google'), 'source_type'] = 'organic'
log.loc[(log.traffic_source == 'paid') | (log.traffic_source == 'email') & (log.region == 'Russia'), 'source_type'] = 'ad'
log.loc[(log.traffic_source == 'paid') | (log.traffic_source == 'email') & (log.region != 'Russia'), 'source_type'] = 'other'

In [7]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


# Задание 2

In [8]:
data = pd.read_csv('URLs.txt')
data.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [9]:
data[data.url.str.contains('/[0-9]{8}-', regex=True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3

In [10]:
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
# считаем общее количество оценок и отсеиваем пользователей, у которых их меньше 100

count = data.groupby('userId').count().reset_index()
count = count[count['rating'] > 100]
count = count[['userId', 'rating']]
count.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [12]:
# максимальное значение timestamp

max_t = data.groupby('userId').max().reset_index()
max_t = max_t[['userId','timestamp']]
max_t.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
max_t.head()

,userId,timestamp_max
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [13]:
# минимальное значение timestamp

min_t = data.groupby('userId').min().reset_index()
min_t = min_t[['userId', 'timestamp']]
min_t.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)
min_t.head()

,userId,timestamp_min
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044


In [14]:
# соединяем все таблицы

new_data = count.merge(max_t, how = 'inner').merge(min_t, how = 'inner')
new_data.head()

,userId,rating,timestamp_max,timestamp_min
0,4,204,949982274,949778714
1,8,116,1154474527,1154389340
2,15,1700,1469330735,997937239
3,17,363,1127476640,1127468587
4,19,423,855195373,855190091


In [15]:
def avg(row):
    avg = row.timestamp_max - row.timestamp_min
    return avg 

new_data['average'] = new_data.apply(avg, axis = 1)
new_data.head()

,userId,rating,timestamp_max,timestamp_min,average
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282


# Задание 4

In [16]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [17]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [18]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [19]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [20]:
# таблица с тремя типами выручки для каждого client_id без указания адреса клиента

without_address = rzd.merge(auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer')

without_address.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [21]:
# таблица с тремя типами выручки для каждого client_id с указанием адреса клиента

with_address = without_address.merge(client_base, on = 'client_id', how = 'outer')

with_address.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
